In [178]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import numpy as np

## Reading the used geojson-file's coordinates into Python

In [2]:
!pip install geojson

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Source for code: https://stackoverflow.com/questions/42753745/how-can-i-parse-geojson-with-python
import geojson
# Change this directory after with open to your relevant directory
with open("C:/Users/viggo/Downloads/GitHub_repos/roetersstraat/abstreet/geojson.json") as square:
    model_boundaries = geojson.load(square)
# Saving the features of the geojson file to a variable
features = model_boundaries['features']

# Only saving the coordinates of the geojson
coordinates_geojson = features[0]["geometry"]["coordinates"][0]
coordinates_geojson

[[4.901983, 52.369094],
 [4.901983, 52.359875],
 [4.923408, 52.359875],
 [4.923408, 52.369094],
 [4.901983, 52.369094]]

In [4]:
# Due to some coordinates appearing twice in the geojson coordinates variable, only unique coordinates will be saved
geojson_coordinates = []
for coordinates in coordinates_geojson:
    if coordinates not in geojson_coordinates:
        geojson_coordinates.append(coordinates)

geojson_coordinates

[[4.901983, 52.369094],
 [4.901983, 52.359875],
 [4.923408, 52.359875],
 [4.923408, 52.369094]]

## Getting exact locations from the Kadasterregister map corresponding to coordinates from geojson

Following a process of pinpointing the geojson coordinates on a Google My Maps-file and creating the square there as well, these coordinates where pinpointed as accurately as humanly possible on the map on the Kadasterregister website (available here: https://bagviewer.kadaster.nl/lvbag/bag-viewer/?zoomlevel=1). This website does not utilise coordinates at all and uses a seperate x-y-axis system. THe following values got out of this system after pinpointing the locations:

In [5]:
# Getting the Kadasterregister x and y-values that are relevant for the geojson-file
bag_coordinates = [[121946.73, 485884.65], [123429.29, 485895.45], [123414.96, 486899.86], [121942.62, 486918.21]]
bag_coordinate = [[121942.00, 485885.00], [123430.00, 486919.00]]

In [6]:
# Setting x and y-values for beginning and ending values
x_begin = 121942.00
x_end = 123430.00
y_begin = 485885.00
y_end = 486919.00

# Setting x_now and y_now
x_now = x_begin
y_now = y_begin

In [7]:
# Looping over all values in between x_begin and x_end and y_begin and y_end, where it loops over all y-values in steps of 1.00 for every x-value
# This method is something which ended up not being used, due to the running time being too much
# while x_now <= x_end:
    # while y_now <= y_end:
        # print([x_now, y_now])

        # y_now += 3.5

    # y_now = y_begin
    # x_now += 3.5

## Getting relevant addresses to scrape from Kadasterregister

I will run a Selenium webbrowser and click every building that fits inside our geojson square. Every new building being clicked, equals a new URL being loaded. All these URLs will be saved into a list and later a CSV-file, so these can be used later wehn scraping all these building blocks or seperate buildings individually.

I have looked into opportunities to scrape this, but this was made extremely difficult by the relevant website and, when finally figured out, requires brute-forcing over 120000 seperate coordinate combos, which each will take at least 7 consecutive days to complete (which is not feasible).

In [38]:
# Setting up a list to track the visited URLs and a set with all object_ids seen so far
visited_urls = []
unique_object_ids = set()

In [29]:
# Thanks ChatGPT for helping me out with this code
driver_manual = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_manual.get("https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012171532&theme=BRT+Achtergrond&geometry.x=121955.8065&geometry.y=485871.435&zoomlevel=15&bijbehorendeAdressen=Pand")

try:
    # Making sure everything still works the way it should
    while True:
        # Getting the current URL
        current_url = driver_manual.current_url

        # Extracting the objectId using string operations when "objectId=" is present in the URL
        if "objectId=" in current_url:
            # Getting the start index of the objectID
            start_idx = current_url.index("objectId=") + len("objectId=")
            # Finding the end index of the objectID
            end_idx = current_url.find("&", start_idx)
            # Handling the case where objectId is the last parameter
            if end_idx == -1: 
                end_idx = len(current_url)
            
            # Getting the current objectID by using the previously found start index and end index of the objectID
            current_object_id = current_url[start_idx:end_idx]

            # Adding the current objectID the set of unique objectIDs if it is a unique one and if it was actually found
            if current_object_id and current_object_id not in unique_object_ids:
                unique_object_ids.add(current_object_id)

                # Adding the URL to the visited URLs list
                visited_urls.append(current_url)

        # Sleeping for a short time to avoid overwhelming the browser
        time.sleep(1)

except KeyboardInterrupt:
    # Stopping the loop gracefully when interrupted
    print("Stopped monitoring.")
    print(f"Visited URLs: {visited_urls}")

finally:
    driver_manual.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x0084EBD3+24307]
	(No symbol) [0x007D8D74]
	(No symbol) [0x006BC323]
	(No symbol) [0x0069E00B]
	(No symbol) [0x00725F6F]
	(No symbol) [0x00738EC9]
	(No symbol) [0x0071FC26]
	(No symbol) [0x006F219C]
	(No symbol) [0x006F311D]
	GetHandleVerifier [0x00AF8D93+2818227]
	GetHandleVerifier [0x00B5542E+3196750]
	GetHandleVerifier [0x00B4D9D2+3165426]
	GetHandleVerifier [0x008EDA70+675216]
	(No symbol) [0x007E1B3D]
	(No symbol) [0x007DEA18]
	(No symbol) [0x007DEBB5]
	(No symbol) [0x007D1640]
	BaseThreadInitThunk [0x766E7BA9+25]
	RtlInitializeExceptionChain [0x7724C0CB+107]
	RtlClearBits [0x7724C04F+191]


In [30]:
print(visited_urls)

['https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012171532&theme=BRT%20Achtergrond&geometry.x=121955.8065&geometry.y=485871.435&zoomlevel=15&bijbehorendeAdressen=Pand', 'https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363010000804487&theme=BRT+Achtergrond&geometry.x=121955.8065&geometry.y=485871.435&zoomlevel=14&bijbehorendeAdressen=Pand', 'https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012183037&theme=BRT+Achtergrond&geometry.x=121959.25349999999&geometry.y=485872.3725&zoomlevel=15', 'https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012183036&theme=BRT+Achtergrond&geometry.x=121984.487&geometry.y=485878.1435&zoomlevel=14&bijbehorendeAdressen=Pand', 'https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012183038&theme=BRT+Achtergrond&geometry.x=122013.684&geometry.y=485885.911&zoomlevel=15&bijbehorendeAdressen=Pand', 'https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363010000804484&theme=BRT+Achtergrond&geom

In [31]:
# Exporting the list to a csv-file, so the result will be saved locally and will not be lost
df_urls = pd.DataFrame(visited_urls)
urls_csv = df_urls.to_csv("visited_urls.csv", index=False)

In [39]:
# Importing the csv-file and reloading it into a list if necesserry
df_urls_d = pd.read_csv("visited_urls.csv")
urls_list = df_urls_d["0"].values.tolist()
print(len(urls_list))

889


## Scraping addresses from the Kadasterregister

In [40]:
# Creating a list with URLs to scrape addresses from
# url_list = ["https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012170489&theme=BRT+Achtergrond&geometry.x=122645.8655&geometry.y=486518.91500000004&zoomlevel=15&bijbehorendeAdressen=Pand", "https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012181651&theme=BRT+Achtergrond&geometry.x=122633.187&geometry.y=486484.1455&zoomlevel=15&bijbehorendeAdressen=Pand", "https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363010000777796&theme=BRT+Achtergrond&geometry.x=122642.67&geometry.y=486478.5025&zoomlevel=15", "https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012180475&theme=BRT+Achtergrond&geometry.x=122595.2835&geometry.y=486528.685&zoomlevel=15&bijbehorendeAdressen=Pand", "https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012170838&theme=BRT+Achtergrond&geometry.x=122834.86050000001&geometry.y=486552.7435&zoomlevel=15&bijbehorendeAdressen=Pand", "https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100012186093&theme=BRT+Achtergrond&geometry.x=122804.3175&geometry.y=486432.181&zoomlevel=14.691288294498632&bijbehorendeAdressen=Pand"]
url_list = urls_list
# Creating an empty list to store the addresses and latitudes and longitudes in
all_scraped_addresses = []

In [41]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in url_list:
    # Getting url from url_list
    driver.get(i)
    
    try:
        # Fetching the sidebar where all address information is present
        sidebar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//app-root//app-sidebar"))
        )

        # Fetching the elements where the name of the single building or building block is located 
        elements_h1 = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//h1"))
        )

        # Running a loop to see if this concerns a building block or seperate building
        # For this part of the loop, it concerns a building block, due to every building block starting with "Pand ID"
        if elements_h1[0].text.strip().startswith("Pand ID"):
            timeout = 15  # Timeout after 15 seconds
            start_time = time.time()

            while time.time() - start_time < timeout:
                h2_elements = driver.find_elements(By.XPATH, "//app-root//app-sidebar//h2")
                if len(h2_elements) >= 5:
                    amount_addresses = h2_elements[1].text  # Assuming the first <h2> is the one you want
                time.sleep(0.5)  # Wait before checking again
        
            # Creating a variable containing only a number for the amount of buildings in the block
            if amount_addresses:
                number_addresses = int(amount_addresses.split()[0])

            # If the amount of addresses is not found, automatically click the back-button so not to click other links
            else:
                print("Unable to determine the number of addresses. Clicking back button.")
                back_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//app-root//app-sidebar//a"))
                )
                back_button.click()
                time.sleep(1)

            # Fetching the addresses if the number of addresses is less than or equal to 5    
            if number_addresses <= 5:
                # Locating the addresses in the sidebar
                buttons_houses = sidebar.find_elements(By.XPATH, "//app-root//app-sidebar//sidebar-overzicht//ul//li//a//span")

                # Looping over all addresses available in the sidebar, to scrape their function and area
                for address in range(0, number_addresses):
                    try:
                        # Re-fetching the buttons_houses after an earlier loop
                        buttons_houses = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//sidebar-overzicht//ul//li//a//span"))
                        )
                        
                        # Creating a list for the scraped variables for the address
                        scraped_text = []

                        # Setting and clicking the button of the address that is relevant for this building
                        rel_address = buttons_houses[address]
                        rel_address.click()

                        # Getting the relevant p-elements for the address
                        elements_p = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//p"))
                        )

                        # Selecting the address here and saving it to the list
                        if len(elements_p) >= 2:
                            scraped_text.append(elements_p[0].text)
                        
                        # Getting the relevant list-elements for the address
                        elements = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//ul//li"))
                        )
                    
                        # Selecting the function and area of the relevant address and saving it to the list
                        if len(elements) >= 4:
                            scraped_text.append(elements[0].text)
                            scraped_text.append(elements[2].text)
                        
                        # Adding the scraped_text list for the building block to the overall-list
                        all_scraped_addresses.append(scraped_text)

                        time.sleep(1)

                        # Finding and clicking the back-button, so the previous addresses can be loaded
                        back_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//app-root//app-sidebar//a"))
                        )
                        back_button.click()

                        time.sleep(1)
                
                    except Exception as e:
                        print(f"Error occurred while processing address {address + 1}: {e}")
                    
            # Fetching the addresses if the number of addresses is bigger than 5
            elif number_addresses > 5:
                # Locating the text connected to the button showing all buildings in the block
                button_more_houses = sidebar.find_elements(By.XPATH, ".//a//span")

                if len(button_more_houses) >= 9:
                    # Clicking the button connected to the previously mentioned text
                    parent_button = button_more_houses[8].find_element(By.XPATH, "./parent::a")
                    parent_button.click()

                time.sleep(1)
        
                # Locating the addresses within the sidebar
                houses_buttons = sidebar.find_elements(By.XPATH, "//app-root//app-sidebar-left//table//tbody//tr//ul//li")
        
                # Looping over all addresses available in the sidebar, to scrape their function and area
                for address in range(0, number_addresses):
                    try:
                        # Creating a list for the scraped variables for the address
                        scraped_text = []

                        # Setting and clicking the button of the address that is relevant for this building
                        rel_address = houses_buttons[address]
                        rel_address.click()

                        time.sleep(1)
                    
                        # Getting the relevant p-elements for the address
                        elements_p = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//p"))
                        )

                        # Selecting the address here and saving it to the list
                        if len(elements_p) >= 2:
                            scraped_text.append(elements_p[0].text)
                        
                        # Getting the relevant list-elements for the address
                        elements = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//ul//li"))
                        )

                        time.sleep(1)
                    
                        # Selecting the function and area of the relevant address and saving it to the list
                        if len(elements) >= 4:
                            scraped_text.append(elements[0].text)
                            scraped_text.append(elements[2].text)
                        
                        # Adding the scraped_text list for the building block to the overall-list
                        all_scraped_addresses.append(scraped_text)

                        time.sleep(1)
                
                    except Exception as e:
                        print(f"Error occurred while processing address {address + 1}: {e}")
            
                # Getting the last address
                last_address = houses_buttons[-1]
                
                # Handling the last address separately (due to an error I kept encountering, where it skipped over the last address)
                # This is done the exact same way is with other addresses
                try:
                    last_address.click()

                    time.sleep(1)

                    WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//p"))
                    )

                    scraped_text = []

                    elements_p = driver.find_elements(By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//p")
                    if len(elements_p) >= 2:
                        scraped_text.append(elements_p[0].text)

                    elements = driver.find_elements(By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//ul//li")

                    if len(elements) >= 4:
                        scraped_text.append(elements[0].text)
                        scraped_text.append(elements[2].text)

                    all_scraped_addresses.append(scraped_text)

                    time.sleep(1)

                except Exception as e:
                    print(f"Error occurred while processing the last address: {e}")
            
        # If the elements_h1[0] part does not start with "Pand ID", then it concerns a seperate building, where the scraping used to get information from seperate addresses is needed
        else:
            try:
                scraped_text = []

                elements_p = driver.find_elements(By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//p")
                scraped_text.append(elements_p[0].text)

                elements = driver.find_elements(By.XPATH, "//app-root//app-sidebar//sidebar-samenvatting//ul//li")

                scraped_text.append(elements[0].text)
                scraped_text.append(elements[3].text)

                all_scraped_addresses.append(scraped_text)

            except Exception as e:
                print(f"Error occurred while processing this address: {e}")
        
        # Pause the search for a bit, so to not overwhelm the website
        time.sleep(1)

    # If no addresses were found or another error occured, the following code will be printed, so to not interrupt the other URLs
    # This will happen quite often if there are buildings which have no address attached to them, such as with: https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100100091821&theme=BRT+Achtergrond&geometry.x=122451.49133918143&geometry.y=486232.7990251022&zoomlevel=14.984798429615658&bijbehorendeAdressen=Pand 
    except Exception as e:
        print(f"Error {e}")

driver.quit()

Error occurred while processing address 1: Message: 

Error occurred while processing address 2: Message: 

Error occurred while processing address 3: Message: 

Error occurred while processing address 4: Message: 

Error occurred while processing address 1: Message: 

Error occurred while processing address 2: Message: 

Error occurred while processing address 3: Message: 

Error occurred while processing address 1: Message: 

Error occurred while processing address 2: Message: 

Error occurred while processing address 1: Message: 

Error occurred while processing address 2: Message: 

Error occurred while processing address 1: Message: 

Error occurred while processing address 2: Message: 

Error occurred while processing address 1: Message: 

Error occurred while processing address 2: Message: 

Error occurred while processing address 1: Message: 

Error occurred while processing address 2: Message: 

Error occurred while processing address 3: Message: 

Error occurred while process

In [ ]:
# Use this code when getting rate-limited (it happened once)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# for i in url_list:
    # Getting url from url_list
    # driver.get(i)
    
    # try:
        # Fetching the sidebar where all address information is present
        # sidebar = WebDriverWait(driver, 10).until(
        # EC.presence_of_element_located((By.XPATH, "//app-root//app-sidebar"))
        # )

        # Fetching the elements where the name of the single building or building block is located 
        # elements_h1 = WebDriverWait(driver, 10).until(
        # EC.presence_of_all_elements_located((By.XPATH, "//app-root//app-sidebar//h1"))
        # )

        # Running a loop to see if this concerns a building block or seperate building
        # For this part of the loop, it concerns a building block, due to every building block starting with "Pand ID"
        # if elements_h1[0].text.strip().startswith("Pand ID"):

            # Find the element mentioning the amount of buildings in that block
            # amount_buildings = sidebar.find_elements(By.XPATH, ".//h2")
        
            # Create a variable containing only a number for the amount of buildings in the block
            # if len(amount_buildings) >= 2:
                # amount_addresses = amount_buildings[1].text
                # number_addresses = int(amount_addresses.split()[0])

            # Fetching the addresses if the number of addresses is less than or equal to 5    
            # if number_addresses <= 5:
                # Locating the addresses in the sidebar
                # buttons_houses = sidebar.find_elements(By.XPATH, "//app-root//app-sidebar//sidebar-overzicht//ul//li//a//span")

                # Adding the addresses to the list into seperate sublists
                # for button in buttons_houses:
                    # address_text = button.text
                    # all_scraped_addresses.append([address_text])
    
            # Fetching the addresses if the number of addresses is bigger than 5
            # elif number_addresses > 5:
                # Locating the text connected to the button showing all buildings in the block
                # button_more_houses = sidebar.find_elements(By.XPATH, ".//a//span")

                # if len(button_more_houses) >= 9:
                    # Clicking the button connected to the previously mentioned text
                    # parent_button = button_more_houses[8].find_element(By.XPATH, "./parent::a")
                    # parent_button.click()
        
                # Locating the addresses within the sidebar
                # houses_buttons = sidebar.find_elements(By.XPATH, "//app-root//app-sidebar-left//table//tbody//tr//ul//li")
        
                # Adding the addresses into to the list into seperate sublists
                # for button in houses_buttons:
                    # address_text = button.text
                    # all_scraped_addresses.append([address_text])

        # If the elements_h1[0] part does not start with "Pand ID", then it concerns a seperate building, where just the title of the building is enough
        # else:
            # all_scraped_addresses.append([elements_h1[0].text])
        
        # Pause the search for a bit, so to not overwhelm the website
        # time.sleep(1)

    # If no addresses were found or another error occured, the following code will be printed, so to not interrupt the other URLs
    # This will happen quite often if there are buildings which have no address attached to them, such as with: https://bagviewer.kadaster.nl/lvbag/bag-viewer/?objectId=0363100100091821&theme=BRT+Achtergrond&geometry.x=122451.49133918143&geometry.y=486232.7990251022&zoomlevel=14.984798429615658&bijbehorendeAdressen=Pand 
    # except Exception as e:
        # print(f"Error {e}")

# driver.quit()

In [42]:
print(all_scraped_addresses)

[['Plantage Kerklaan 15-2\n1018 SZ  AMSTERDAM', 'Woonfunctie', '55m²'], ['Plantage Kerklaan 15-3\n1018 SZ  AMSTERDAM', 'Woonfunctie', '46m²'], ['Plantage Kerklaan 15 H\n1018 SZ  AMSTERDAM', 'Woonfunctie', '146m²'], ['Plantage Kerklaan 17-2\n1018 SZ  AMSTERDAM', 'Woonfunctie', '139m²'], ['Plantage Kerklaan 17 H\n1018 SZ  AMSTERDAM', 'Woonfunctie', '110m²'], ['Plantage Kerklaan 19\n1018 SZ  AMSTERDAM', 'Logiesfunctie', '352m²'], ['Plantage Kerklaan 21 A\n1018 SZ  AMSTERDAM', 'Winkelfunctie', '103m²'], ['Plantage Kerklaan 21 B\n1018 SZ  AMSTERDAM', 'Woonfunctie', '39m²'], ['Plantage Kerklaan 21 C\n1018 SZ  AMSTERDAM', 'Woonfunctie', '64m²'], ['Plantage Kerklaan 23-2\n1018 SZ  AMSTERDAM', 'Woonfunctie', '120m²'], ['Plantage Kerklaan 23 H\n1018 SZ  AMSTERDAM', 'Woonfunctie', '123m²'], ['Plantage Middenlaan 28 H\n1018 DE  AMSTERDAM', 'Winkelfunctie', '60m²'], ['Plantage Middenlaan 28 A\n1018 DE  AMSTERDAM', 'Woonfunctie', '84m²'], ['Plantage Middenlaan 28 B\n1018 DE  AMSTERDAM', 'Woonfunctie

In [43]:
print(len(all_scraped_addresses))

4369


## Using geopy to fill the list with further info needed for each building 

In [44]:
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable


In [14]:
# Test for using geopy on the dataset
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="scraping_addresses.ipynb")
location = geolocator.geocode("Roetersstraat 170, Amsterdam")
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

Kriterion, 170, Roetersstraat, Weesperbuurt, Centrum, Amsterdam, Noord-Holland, Nederland, 1018 WE, Nederland
(52.362493, 4.9106652)
{'place_id': 144946830, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 253130481, 'lat': '52.362493', 'lon': '4.9106652', 'class': 'amenity', 'type': 'cinema', 'place_rank': 30, 'importance': 0.2522941968270874, 'addresstype': 'amenity', 'name': 'Kriterion', 'display_name': 'Kriterion, 170, Roetersstraat, Weesperbuurt, Centrum, Amsterdam, Noord-Holland, Nederland, 1018 WE, Nederland', 'boundingbox': ['52.3624430', '52.3625430', '4.9106152', '4.9107152']}


In [45]:
# Looping over all previously added addresses and getting their relevant latitudes, longitudes and function
# Source for code: https://stackoverflow.com/questions/5807195/how-to-get-coordinates-of-address-from-python
for sublist in all_scraped_addresses:
    address = sublist[0]
    try:
        location = geolocator.geocode(address)
        if location:
            # Add latitude, longitude and function to the sublist
            sublist.append(location.latitude)
            sublist.append(location.longitude)
            sublist.append(location.raw["type"])
            # Adding the name of the building if it is present, otherwise add None to the sublist
            if location.raw["name"] == "":
                sublist.append(None)
            else:
                sublist.append(location.raw["name"])
        else:
            # Appending None if the location is not found
            sublist.append(None)
            sublist.append(None)
            sublist.append(None)
            sublist.append(None)
        
        # Adding a sleep time to comply towards usage policies
        time.sleep(1)


    except Exception as e:
        print(f"Error occurred for address {address}: {e}")
        

In [46]:
print(all_scraped_addresses)

[['Plantage Kerklaan 15-2\n1018 SZ  AMSTERDAM', 'Woonfunctie', '55m²', 52.3660751, 4.9111962, 'house', None], ['Plantage Kerklaan 15-3\n1018 SZ  AMSTERDAM', 'Woonfunctie', '46m²', 52.3660752, 4.911196, 'house', None], ['Plantage Kerklaan 15 H\n1018 SZ  AMSTERDAM', 'Woonfunctie', '146m²', 52.3660753, 4.9111958, 'house', None], ['Plantage Kerklaan 17-2\n1018 SZ  AMSTERDAM', 'Woonfunctie', '139m²', 52.3661293, 4.9112544, 'house', None], ['Plantage Kerklaan 17 H\n1018 SZ  AMSTERDAM', 'Woonfunctie', '110m²', 52.3661293, 4.9112542, 'house', None], ['Plantage Kerklaan 19\n1018 SZ  AMSTERDAM', 'Logiesfunctie', '352m²', 52.3661715, 4.9112673, 'hostel', 'The Bee Hostel'], ['Plantage Kerklaan 21 A\n1018 SZ  AMSTERDAM', 'Winkelfunctie', '103m²', 52.3662016, 4.9113564, 'house', None], ['Plantage Kerklaan 21 B\n1018 SZ  AMSTERDAM', 'Woonfunctie', '39m²', 52.3662017, 4.9113562, 'house', None], ['Plantage Kerklaan 21 C\n1018 SZ  AMSTERDAM', 'Woonfunctie', '64m²', 52.3662018, 4.911356, 'house', None], 

In [ ]:
# Exporting the list to a csv-file, so the result will be saved locally and will not be lost
df_scraped_addresses_3 = pd.DataFrame(all_scraped_addresses)
df_scraped_addresses_3.to_csv("all_scraped_addresses_3.csv", index=False)

### Importing the file with all_scraped_addresses and cleaning the data

In [277]:
# Importing the csv-file and reloading it into a list if necesserry
df_addresses = pd.read_csv("all_scraped_addresses_uncleaned.csv", encoding="iso-8859-1", delimiter=";")
df_addresses

,ï»¿0,1,2,3,4,5,6
0,Sarphatistraat 4\n1017 WS AMSTERDAM,Kantoorfunctie,760mÂ²,52.359783,4.902103,house,NaN
1,Sarphatistraat 6\n1017 WS AMSTERDAM,Kantoorfunctie,1875,52.359801,4.902221,house,NaN
2,Sarphatistraat 6\n1017 WS AMSTERDAM,Kantoorfunctie,Verblijfsobject in gebruik,52.359801,4.902221,house,NaN
3,Sarphatistraat 8\n1017 WS AMSTERDAM,Bijeenkomstfunctie,1880,52.359820,4.902338,house,NaN
4,Sarphatistraat 12\n1017 WS AMSTERDAM,Gezondheidszorgfunctie,1249mÂ²,52.359856,4.902558,house,NaN
...,...,...,...,...,...,...,...
9773,Nieuwe Achtergracht 108-2\n1018 WT AMSTERDAM,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9774,Nieuwe Achtergracht 108-3\n1018 WT AMSTERDAM,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9775,Nieuwe Achtergracht 108 H\n1018 WT AMSTERDAM,Industriefunctie,136mÂ²,52.362644,4.909648,house,NaN
9776,Nieuwe Achtergracht 108 H\n1018 WT AMSTERDAM,Industriefunctie,136mÂ²,52.362644,4.909648,house,NaN


In [278]:
# Giving all columns logical names
df_addresses.columns = ["Address", "Function", "Size", "Latitude", "Longitude", "Category", "Name"]

df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name
0,Sarphatistraat 4\n1017 WS AMSTERDAM,Kantoorfunctie,760mÂ²,52.359783,4.902103,house,NaN
1,Sarphatistraat 6\n1017 WS AMSTERDAM,Kantoorfunctie,1875,52.359801,4.902221,house,NaN
2,Sarphatistraat 6\n1017 WS AMSTERDAM,Kantoorfunctie,Verblijfsobject in gebruik,52.359801,4.902221,house,NaN
3,Sarphatistraat 8\n1017 WS AMSTERDAM,Bijeenkomstfunctie,1880,52.359820,4.902338,house,NaN
4,Sarphatistraat 12\n1017 WS AMSTERDAM,Gezondheidszorgfunctie,1249mÂ²,52.359856,4.902558,house,NaN
...,...,...,...,...,...,...,...
9773,Nieuwe Achtergracht 108-2\n1018 WT AMSTERDAM,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9774,Nieuwe Achtergracht 108-3\n1018 WT AMSTERDAM,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9775,Nieuwe Achtergracht 108 H\n1018 WT AMSTERDAM,Industriefunctie,136mÂ²,52.362644,4.909648,house,NaN
9776,Nieuwe Achtergracht 108 H\n1018 WT AMSTERDAM,Industriefunctie,136mÂ²,52.362644,4.909648,house,NaN


In [279]:
# Filtering out all duplicates for the Address column (due to some duplicates having a unique value for Size column)
df_addresses = df_addresses.drop_duplicates(subset="Address", keep="first")
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name
0,Sarphatistraat 4\n1017 WS AMSTERDAM,Kantoorfunctie,760mÂ²,52.359783,4.902103,house,NaN
1,Sarphatistraat 6\n1017 WS AMSTERDAM,Kantoorfunctie,1875,52.359801,4.902221,house,NaN
3,Sarphatistraat 8\n1017 WS AMSTERDAM,Bijeenkomstfunctie,1880,52.359820,4.902338,house,NaN
4,Sarphatistraat 12\n1017 WS AMSTERDAM,Gezondheidszorgfunctie,1249mÂ²,52.359856,4.902558,house,NaN
5,Sarphatistraat 14\n1017 WS AMSTERDAM,Kantoorfunctie,516mÂ²,52.359856,4.902558,house,NaN
...,...,...,...,...,...,...,...
9772,Nieuwe Achtergracht 108-1\n1018 WT AMSTERDAM,Woonfunctie,55mÂ²,52.362645,4.909648,house,NaN
9773,Nieuwe Achtergracht 108-2\n1018 WT AMSTERDAM,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9774,Nieuwe Achtergracht 108-3\n1018 WT AMSTERDAM,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9775,Nieuwe Achtergracht 108 H\n1018 WT AMSTERDAM,Industriefunctie,136mÂ²,52.362644,4.909648,house,NaN


In [280]:
# Deleting the ZIP-code and city from the Address-column by deleting everything after the "\n"-character
df_addresses["Address"] = df_addresses["Address"].str.split("\n").str[0]
df_addresses

C:\Users\viggo\AppData\Local\Temp\ipykernel_2928\295733086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_addresses["Address"] = df_addresses["Address"].str.split("\n").str[0]


,Address,Function,Size,Latitude,Longitude,Category,Name
0,Sarphatistraat 4,Kantoorfunctie,760mÂ²,52.359783,4.902103,house,NaN
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,house,NaN
3,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,house,NaN
4,Sarphatistraat 12,Gezondheidszorgfunctie,1249mÂ²,52.359856,4.902558,house,NaN
5,Sarphatistraat 14,Kantoorfunctie,516mÂ²,52.359856,4.902558,house,NaN
...,...,...,...,...,...,...,...
9772,Nieuwe Achtergracht 108-1,Woonfunctie,55mÂ²,52.362645,4.909648,house,NaN
9773,Nieuwe Achtergracht 108-2,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9774,Nieuwe Achtergracht 108-3,Woonfunctie,55mÂ²,52.362644,4.909648,house,NaN
9775,Nieuwe Achtergracht 108 H,Industriefunctie,136mÂ²,52.362644,4.909648,house,NaN


In [281]:
# Creating a seperate DataFrame containing all rows where the Address-column starts with "Voorm Stadstimmertuin"
df_voorm_stadstimmertuin = df_addresses[df_addresses["Address"].str.startswith("Voorm Stadstimmertuin")]

df_voorm_stadstimmertuin

,Address,Function,Size,Latitude,Longitude,Category,Name
9055,Voorm Stadstimmertuin 91,Industriefunctie,475mÂ²,NaN,NaN,NaN,NaN
9056,Voorm Stadstimmertuin 35,Woonfunctie,44mÂ²,NaN,NaN,NaN,NaN
9058,Voorm Stadstimmertuin 37,Woonfunctie,48mÂ²,NaN,NaN,NaN,NaN
9060,Voorm Stadstimmertuin 39,Woonfunctie,44mÂ²,NaN,NaN,NaN,NaN
9062,Voorm Stadstimmertuin 41,Woonfunctie,48mÂ²,NaN,NaN,NaN,NaN
9064,Voorm Stadstimmertuin 43,Woonfunctie,66mÂ²,NaN,NaN,NaN,NaN
9066,Voorm Stadstimmertuin 45,Woonfunctie,70mÂ²,NaN,NaN,NaN,NaN
9068,Voorm Stadstimmertuin 47 A,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN
9069,Voorm Stadstimmertuin 47 B,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN
9070,Voorm Stadstimmertuin 47 C,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN


In [282]:
# This street was completely skipped over by Geopy due to it not having its complete name here
# Changing "Voorm Stadstimmertuin" to "Voormalige Stadstimmertuin"
df_voorm_stadstimmertuin["Address"] = df_voorm_stadstimmertuin["Address"].str.replace("Voorm Stadstimmertuin", "Voormalige Stadstimmertuin", regex=False)

df_voorm_stadstimmertuin

C:\Users\viggo\AppData\Local\Temp\ipykernel_2928\2333366102.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_voorm_stadstimmertuin["Address"] = df_voorm_stadstimmertuin["Address"].str.replace("Voorm Stadstimmertuin", "Voormalige Stadstimmertuin", regex=False)


,Address,Function,Size,Latitude,Longitude,Category,Name
9055,Voormalige Stadstimmertuin 91,Industriefunctie,475mÂ²,NaN,NaN,NaN,NaN
9056,Voormalige Stadstimmertuin 35,Woonfunctie,44mÂ²,NaN,NaN,NaN,NaN
9058,Voormalige Stadstimmertuin 37,Woonfunctie,48mÂ²,NaN,NaN,NaN,NaN
9060,Voormalige Stadstimmertuin 39,Woonfunctie,44mÂ²,NaN,NaN,NaN,NaN
9062,Voormalige Stadstimmertuin 41,Woonfunctie,48mÂ²,NaN,NaN,NaN,NaN
9064,Voormalige Stadstimmertuin 43,Woonfunctie,66mÂ²,NaN,NaN,NaN,NaN
9066,Voormalige Stadstimmertuin 45,Woonfunctie,70mÂ²,NaN,NaN,NaN,NaN
9068,Voormalige Stadstimmertuin 47 A,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN
9069,Voormalige Stadstimmertuin 47 B,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN
9070,Voormalige Stadstimmertuin 47 C,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN


In [283]:
# Converting the DataFrame to an array
array_voorm_stadstimmertuin = df_voorm_stadstimmertuin.values.tolist()

print(array_voorm_stadstimmertuin)

[['Voormalige Stadstimmertuin 91', 'Industriefunctie', '475mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 35', 'Woonfunctie', '44mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 37', 'Woonfunctie', '48mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 39', 'Woonfunctie', '44mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 41', 'Woonfunctie', '48mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 43', 'Woonfunctie', '66mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 45', 'Woonfunctie', '70mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 47 A', 'Woonfunctie', '35mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 47 B', 'Woonfunctie', '35mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 47 C', 'Woonfunctie', '35mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 47 D', 'Woonfunctie', '35mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 47 E', 'Woonfunctie', '35mÂ²', nan, nan, nan, nan], ['Voormalige Stadstimmertuin 47 F',

In [284]:
# Using the previously defined code used for adding the Geopy columns
for sublist in array_voorm_stadstimmertuin:
    address = sublist[0]
    try:
        location = geolocator.geocode(address)
        if location:
            # Add latitude, longitude and function to the sublist
            sublist.append(location.latitude)
            sublist.append(location.longitude)
            sublist.append(location.raw["type"])
            # Adding the name of the building if it is present, otherwise add None to the sublist
            if location.raw["name"] == "":
                sublist.append(None)
            else:
                sublist.append(location.raw["name"])
        else:
            # Appending None if the location is not found
            sublist.append(None)
            sublist.append(None)
            sublist.append(None)
            sublist.append(None)
        
        # Adding a sleep time to comply towards usage policies
        time.sleep(1)


    except Exception as e:
        print(f"Error occurred for address {address}: {e}")

In [285]:
array_voorm_stadstimmertuin

[['Voormalige Stadstimmertuin 91',
  'Industriefunctie',
  '475mÂ²',
  nan,
  nan,
  nan,
  nan,
  52.3617056,
  4.9063691,
  'house',
  None],
 ['Voormalige Stadstimmertuin 35',
  'Woonfunctie',
  '44mÂ²',
  nan,
  nan,
  nan,
  nan,
  52.3615502,
  4.9057395,
  'house',
  None],
 ['Voormalige Stadstimmertuin 37',
  'Woonfunctie',
  '48mÂ²',
  nan,
  nan,
  nan,
  nan,
  52.3615593,
  4.9057688,
  'house',
  None],
 ['Voormalige Stadstimmertuin 39',
  'Woonfunctie',
  '44mÂ²',
  nan,
  nan,
  nan,
  nan,
  52.3615684,
  4.9057834,
  'house',
  None],
 ['Voormalige Stadstimmertuin 41',
  'Woonfunctie',
  '48mÂ²',
  nan,
  nan,
  nan,
  nan,
  52.3615685,
  4.9058127,
  'house',
  None],
 ['Voormalige Stadstimmertuin 43',
  'Woonfunctie',
  '66mÂ²',
  nan,
  nan,
  nan,
  nan,
  52.3615775,
  4.9058273,
  'house',
  None],
 ['Voormalige Stadstimmertuin 45',
  'Woonfunctie',
  '70mÂ²',
  nan,
  nan,
  nan,
  nan,
  52.3615776,
  4.9058567,
  'house',
  None],
 ['Voormalige Stadstimmertui

In [286]:
# Converting the array back to a DataFrame
df_voormalige_stadstimmertuin = pd.DataFrame(array_voorm_stadstimmertuin, columns=["Address", "Function", "Size", "No_1", "No_2", "No_3", "No_4", "Latitude", "Longitude", "Category", "Name"])

df_voormalige_stadstimmertuin

,Address,Function,Size,No_1,No_2,No_3,No_4,Latitude,Longitude,Category,Name
0,Voormalige Stadstimmertuin 91,Industriefunctie,475mÂ²,NaN,NaN,NaN,NaN,52.361706,4.906369,house,None
1,Voormalige Stadstimmertuin 35,Woonfunctie,44mÂ²,NaN,NaN,NaN,NaN,52.361550,4.905740,house,None
2,Voormalige Stadstimmertuin 37,Woonfunctie,48mÂ²,NaN,NaN,NaN,NaN,52.361559,4.905769,house,None
3,Voormalige Stadstimmertuin 39,Woonfunctie,44mÂ²,NaN,NaN,NaN,NaN,52.361568,4.905783,house,None
4,Voormalige Stadstimmertuin 41,Woonfunctie,48mÂ²,NaN,NaN,NaN,NaN,52.361568,4.905813,house,None
5,Voormalige Stadstimmertuin 43,Woonfunctie,66mÂ²,NaN,NaN,NaN,NaN,52.361578,4.905827,house,None
6,Voormalige Stadstimmertuin 45,Woonfunctie,70mÂ²,NaN,NaN,NaN,NaN,52.361578,4.905857,house,None
7,Voormalige Stadstimmertuin 47 A,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN,52.361623,4.905988,house,None
8,Voormalige Stadstimmertuin 47 B,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN,52.361623,4.905988,house,None
9,Voormalige Stadstimmertuin 47 C,Woonfunctie,35mÂ²,NaN,NaN,NaN,NaN,52.361624,4.905988,house,None


In [287]:
columns_to_delete = ["No_1", "No_2", "No_3", "No_4"]
df_voormalige_stadstimmertuin = df_voormalige_stadstimmertuin.drop(columns=columns_to_delete)

df_voormalige_stadstimmertuin

,Address,Function,Size,Latitude,Longitude,Category,Name
0,Voormalige Stadstimmertuin 91,Industriefunctie,475mÂ²,52.361706,4.906369,house,None
1,Voormalige Stadstimmertuin 35,Woonfunctie,44mÂ²,52.361550,4.905740,house,None
2,Voormalige Stadstimmertuin 37,Woonfunctie,48mÂ²,52.361559,4.905769,house,None
3,Voormalige Stadstimmertuin 39,Woonfunctie,44mÂ²,52.361568,4.905783,house,None
4,Voormalige Stadstimmertuin 41,Woonfunctie,48mÂ²,52.361568,4.905813,house,None
5,Voormalige Stadstimmertuin 43,Woonfunctie,66mÂ²,52.361578,4.905827,house,None
6,Voormalige Stadstimmertuin 45,Woonfunctie,70mÂ²,52.361578,4.905857,house,None
7,Voormalige Stadstimmertuin 47 A,Woonfunctie,35mÂ²,52.361623,4.905988,house,None
8,Voormalige Stadstimmertuin 47 B,Woonfunctie,35mÂ²,52.361623,4.905988,house,None
9,Voormalige Stadstimmertuin 47 C,Woonfunctie,35mÂ²,52.361624,4.905988,house,None


In [288]:
# Putting the df_voormalige_stadstimmertuin and df_addresses in the same DataFrame
df_addresses = pd.concat([df_addresses, df_voormalige_stadstimmertuin], ignore_index=True)
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name
0,Sarphatistraat 4,Kantoorfunctie,760mÂ²,52.359783,4.902103,house,NaN
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,house,NaN
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,house,NaN
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249mÂ²,52.359856,4.902558,house,NaN
4,Sarphatistraat 14,Kantoorfunctie,516mÂ²,52.359856,4.902558,house,NaN
...,...,...,...,...,...,...,...
9291,Voormalige Stadstimmertuin 2 K,Woonfunctie,184mÂ²,52.361258,4.905530,house,None
9292,Voormalige Stadstimmertuin 2 L,Woonfunctie,139mÂ²,52.361189,4.905337,house,None
9293,Voormalige Stadstimmertuin 2 M,Woonfunctie,169mÂ²,52.361222,4.905546,house,None
9294,Voormalige Stadstimmertuin 2 N,Woonfunctie,446mÂ²,52.361169,4.905344,house,None


In [289]:
# Deleting all rows where no coordinates are present
df_addresses = df_addresses[df_addresses["Latitude"].notna() & (df_addresses["Latitude"] != "")]
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name
0,Sarphatistraat 4,Kantoorfunctie,760mÂ²,52.359783,4.902103,house,NaN
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,house,NaN
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,house,NaN
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249mÂ²,52.359856,4.902558,house,NaN
4,Sarphatistraat 14,Kantoorfunctie,516mÂ²,52.359856,4.902558,house,NaN
...,...,...,...,...,...,...,...
9291,Voormalige Stadstimmertuin 2 K,Woonfunctie,184mÂ²,52.361258,4.905530,house,None
9292,Voormalige Stadstimmertuin 2 L,Woonfunctie,139mÂ²,52.361189,4.905337,house,None
9293,Voormalige Stadstimmertuin 2 M,Woonfunctie,169mÂ²,52.361222,4.905546,house,None
9294,Voormalige Stadstimmertuin 2 N,Woonfunctie,446mÂ²,52.361169,4.905344,house,None


In [290]:
# Changing the category-column value towards the correct category following the scraped Function-column
# The Function-column is more trust-worthy, due to it being directly scraped from the Kadasterregistry, an official dataset for all buildings in the Netherlands
df_addresses.loc[(df_addresses["Function"] == "Kantoorfunctie") & (df_addresses["Category"] == "house"), "Category"] = "office"
df_addresses.loc[(df_addresses["Function"] == "Kantoorfunctie") & (df_addresses["Category"] == "yes"), "Category"] = "office"
df_addresses.loc[(df_addresses["Function"] == "Bijeenkomstfunctie") & (df_addresses["Category"] == "house"), "Category"] = "public_place"
df_addresses.loc[(df_addresses["Function"] == "Bijeenkomstfunctie") & (df_addresses["Category"] == "tertiary"), "Category"] = "restaurant"
df_addresses.loc[(df_addresses["Address"] == "Mauritskade 61"), "Category"] = "hotel"
df_addresses.loc[(df_addresses["Address"] == "Henri Polaklaan 9"), "Category"] = "public_place"
df_addresses.loc[(df_addresses["Address"] == "Tweede Boerhaavestraat 80"), "Category"] = "homeless_shelter"
df_addresses.loc[(df_addresses["Address"] == "Plantage Doklaan 16"), "Category"] = "homeless_shelter"
df_addresses.loc[(df_addresses["Address"] == "Roetersstraat 2"), "Category"] = "old_people_home"
df_addresses.loc[(df_addresses["Address"] == "Lepelstraat 2"), "Category"] = "house"
df_addresses.loc[(df_addresses["Function"] == "Gezondheidszorgfunctie") & (df_addresses["Category"] == "house"), "Category"] = "health_care"
df_addresses.loc[(df_addresses["Function"] == "Gezondheidszorgfunctie") & (df_addresses["Category"] == "residential"), "Category"] = "health_care"
df_addresses.loc[(df_addresses["Function"] == "Logiesfunctie") & (df_addresses["Category"] == "house"), "Category"] = "hotel"
df_addresses.loc[(df_addresses["Function"] == "Logiesfunctie") & (df_addresses["Category"] == "apartment"), "Category"] = "hotel"
df_addresses.loc[(df_addresses["Function"] == "Logiesfunctie") & (df_addresses["Category"] == "toilets"), "Category"] = "hotel"
df_addresses.loc[(df_addresses["Function"] == "Logiesfunctie") & (df_addresses["Category"] == "residential"), "Category"] = "hotel"
df_addresses.loc[(df_addresses["Address"] == "Waterlooplein 211"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Waterlooplein 213"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Waterlooplein 211"), "Name"] = "Academie van de Bouwkunst"
df_addresses.loc[(df_addresses["Address"] == "Waterlooplein 213"), "Name"] = "Academie van de Bouwkunst"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Herengracht 103"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Herengracht 103"), "Name"] = "Amsterdam Fashion Academy"
df_addresses.loc[(df_addresses["Address"] == "Mauritskade 58"), "Category"] = "secondary_school"
df_addresses.loc[(df_addresses["Address"] == "Mauritskade 11"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Mauritskade 11"), "Name"] = "HvA Koetsier-Montaignehuis"
df_addresses.loc[(df_addresses["Address"] == "Rhijnspoorplein 2"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Rhijnspoorplein 2"), "Name"] = "HvA Jakoba Mulder-huis"
df_addresses.loc[(df_addresses["Address"] == "Wibautstraat 2"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Wibautstraat 2"), "Name"] = "HvA Kohnstammhuis"
df_addresses.loc[(df_addresses["Address"] == "Wibautstraat 4"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Wibautstraat 4"), "Name"] = "HvA Kohnstammhuis"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33"), "Name"] = "Studio Hogeschool van Amsterdam"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 A"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 A"), "Name"] = "Studio Hogeschool van Amsterdam"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 B"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 B"), "Name"] = "Studio Hogeschool van Amsterdam"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 C"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 C"), "Name"] = "Studio Hogeschool van Amsterdam"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 D"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 D"), "Name"] = "Studio Hogeschool van Amsterdam"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 E"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 E"), "Name"] = "Studio Hogeschool van Amsterdam"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 F"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Eerste Boerhaavestraat 33 F"), "Name"] = "Studio Hogeschool van Amsterdam"
df_addresses.loc[(df_addresses["Address"] == "Plantage Middenlaan 2"), "Category"] = "restaurant"
df_addresses.loc[(df_addresses["Address"] == "Plantage Muidergracht 12"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Plantage Muidergracht 12"), "Name"] = "UvA REC M"
df_addresses.loc[(df_addresses["Address"] == "Plantage Muidergracht 14"), "Category"] = "office"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Kerkstraat 126"), "Category"] = "childcare"
df_addresses.loc[(df_addresses["Address"] == "Korte Lepelstraat 81"), "Category"] = "primary_school"
df_addresses.loc[(df_addresses["Address"] == "Korte Lepelstraat 81"), "Name"] = "OBS Dr. E. Boekmanschool"
df_addresses.loc[(df_addresses["Address"] == "Roetersstraat 11"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Roetersstraat 11"), "Name"] = "UvA REC E"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 127"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 127"), "Name"] = "UvA REC De Brug"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 A"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 A"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 B"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 B"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 C"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 C"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 D"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 D"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 E"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 E"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Prinsengracht 130"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Prinsengracht 130"), "Name"] = "UvA REC G"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 166"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 166"), "Name"] = "UvA REC A"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 160"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 160"), "Name"] = "UvA REC A/B/C Main Hall"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 A"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 A"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 B"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 B"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 C"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 C"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 D"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 D"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 E"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129 E"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Prinsengracht 130"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Prinsengracht 130"), "Name"] = "UvA REC G"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 166"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 166"), "Name"] = "UvA REC A"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 160"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 160"), "Name"] = "UvA REC ABC"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 129"), "Name"] = "UvA REC LAB"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 170"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Achtergracht 170"), "Name"] = "UvA REC CREA"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 59"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 59"), "Name"] = "UvA REC V"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 61"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 61"), "Name"] = "UvA REC V"
df_addresses.loc[(df_addresses["Address"] == "Weesperplein 8"), "Category"] = "office"
df_addresses.loc[(df_addresses["Address"] == "Weesperplein 8"), "Name"] = "Municipality"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 67"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 67"), "Name"] = "UvA REC J/K"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 65"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Valckenierstraat 65"), "Name"] = "UvA REC J/K"
df_addresses.loc[(df_addresses["Address"] == "Plantage Muidergracht 24"), "Category"] = "college"
df_addresses.loc[(df_addresses["Address"] == "Plantage Muidergracht 24"), "Name"] = "UvA REC P"
df_addresses.loc[(df_addresses["Function"] == "Sportfunctie") & (df_addresses["Category"] == "house"), "Category"] = "sport"
df_addresses.loc[(df_addresses["Function"] == "Winkelfunctie") & (df_addresses["Category"] == "house"), "Category"] = "shop"
df_addresses.loc[(df_addresses["Function"] == "Winkelfunctie") & (df_addresses["Category"] == "unclassified"), "Category"] = "shop"
df_addresses.loc[(df_addresses["Function"] == "Woonfunctie") & (df_addresses["Category"] == "unclassified"), "Category"] = "house"
df_addresses.loc[(df_addresses["Function"] == "Woonfunctie") & (df_addresses["Category"] == "yes"), "Category"] = "house"
df_addresses.loc[(df_addresses["Function"] == "Woonfunctie") & (df_addresses["Category"] == "tertiary"), "Category"] = "house"
df_addresses.loc[(df_addresses["Category"] == "residential"), "Category"] = "house"
df_addresses.loc

df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name
0,Sarphatistraat 4,Kantoorfunctie,760mÂ²,52.359783,4.902103,office,NaN
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,office,NaN
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,public_place,NaN
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249mÂ²,52.359856,4.902558,health_care,NaN
4,Sarphatistraat 14,Kantoorfunctie,516mÂ²,52.359856,4.902558,office,NaN
...,...,...,...,...,...,...,...
9291,Voormalige Stadstimmertuin 2 K,Woonfunctie,184mÂ²,52.361258,4.905530,house,None
9292,Voormalige Stadstimmertuin 2 L,Woonfunctie,139mÂ²,52.361189,4.905337,house,None
9293,Voormalige Stadstimmertuin 2 M,Woonfunctie,169mÂ²,52.361222,4.905546,house,None
9294,Voormalige Stadstimmertuin 2 N,Woonfunctie,446mÂ²,52.361169,4.905344,house,None


In [291]:
print(df_addresses["Category"].value_counts())

Category
house               8562
office               302
public_place         100
shop                  67
hotel                 43
                    ... 
secondary_school       1
fitness_centre         1
bar                    1
school                 1
health_centre          1
Name: count, Length: 68, dtype: int64


In [292]:
# Cleaning up the Category-column even more
df_addresses.loc[(df_addresses["Category"] == "company"), "Category"] = "office"
df_addresses.loc[(df_addresses["Category"] == "it"), "Category"] = "office"
df_addresses.loc[(df_addresses["Category"] == "foundation"), "Category"] = "office"
df_addresses.loc[(df_addresses["Category"] == "ngo"), "Category"] = "office"

In [293]:
# Creating a filtered DataFrame for the buildings with an Unclassified function
df_unclassified = df_addresses[df_addresses["Category"] == "unclassified"]
df_unclassified

,Address,Function,Size,Latitude,Longitude,Category,Name
6123,Nieuwe Kerkstraat 1 CH,Overige gebruiksfunctie,107mÂ²,52.364236,4.905878,unclassified,Nieuwe Kerkstraat
7259,Lepelstraat 2 CH,Industriefunctie,63mÂ²,52.362679,4.905598,unclassified,Lepelstraat


In [294]:
# Changing these Categories towards my best guess of their function
df_addresses.loc[(df_addresses["Address"] == "Nieuwe Kerkstraat 1 CH") & (df_addresses["Category"] == "unclassified"), "Category"] = "house"
df_addresses.loc[(df_addresses["Address"] == "Lepelstraat 2 CH") & (df_addresses["Category"] == "unclassified"), "Category"] = "house"

In [295]:
df_addresses.to_csv("scraped_addresses_changing.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'scraped_addresses_changing.csv'

In [337]:
# Importing a manually changed dataset, so to have a useful dataset for the size column
df_addresses = pd.read_csv("scraped_addresses_changing.csv", delimiter=";")
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name
0,Sarphatistraat 4,Kantoorfunctie,760-/-/-/-/,52.359783,4.902103,office,NaN
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,office,NaN
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,public_place,NaN
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249-/-/-/-/,52.359856,4.902558,health_care,NaN
4,Sarphatistraat 14,Kantoorfunctie,516-/-/-/-/,52.359856,4.902558,office,NaN
...,...,...,...,...,...,...,...
9232,Voormalige Stadstimmertuin 2 K,Woonfunctie,184-/-/-/-/,52.361258,4.905530,house,NaN
9233,Voormalige Stadstimmertuin 2 L,Woonfunctie,139-/-/-/-/,52.361189,4.905337,house,NaN
9234,Voormalige Stadstimmertuin 2 M,Woonfunctie,169-/-/-/-/,52.361222,4.905546,house,NaN
9235,Voormalige Stadstimmertuin 2 N,Woonfunctie,446-/-/-/-/,52.361169,4.905344,house,NaN


In [338]:
# Creating a function to delete the "-/-/-/-/" from the Size-column, after making all values not containing this NaN
def remove_m2(column):
    # Only processing strings, so to not confuse the code
    if isinstance(column, str):
        # If the value does not contain "-/-/-/-/", return NaN
        if "-/-/-/-/" not in column:
            return np.nan
        # Replacing "-/-/-/-/" with nothing, so only the area remains
        column = column.replace("-/-/-/-/", "").strip()
        return column if column.isdigit() else np.nan
    # Returning NaN for non-strings
    return np.nan

In [339]:
# Applying the remove_m2 function on the Size-column
df_addresses["Area"] = df_addresses["Size"].apply(remove_m2)
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name,Area
0,Sarphatistraat 4,Kantoorfunctie,760-/-/-/-/,52.359783,4.902103,office,NaN,760
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,office,NaN,NaN
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,public_place,NaN,NaN
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249-/-/-/-/,52.359856,4.902558,health_care,NaN,1249
4,Sarphatistraat 14,Kantoorfunctie,516-/-/-/-/,52.359856,4.902558,office,NaN,516
...,...,...,...,...,...,...,...,...
9232,Voormalige Stadstimmertuin 2 K,Woonfunctie,184-/-/-/-/,52.361258,4.905530,house,NaN,184
9233,Voormalige Stadstimmertuin 2 L,Woonfunctie,139-/-/-/-/,52.361189,4.905337,house,NaN,139
9234,Voormalige Stadstimmertuin 2 M,Woonfunctie,169-/-/-/-/,52.361222,4.905546,house,NaN,169
9235,Voormalige Stadstimmertuin 2 N,Woonfunctie,446-/-/-/-/,52.361169,4.905344,house,NaN,446


In [340]:
# Converting all values in the Area-column to integers
df_addresses["Area"] = df_addresses["Area"].astype("Int64")
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name,Area
0,Sarphatistraat 4,Kantoorfunctie,760-/-/-/-/,52.359783,4.902103,office,NaN,760
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,office,NaN,<NA>
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,public_place,NaN,<NA>
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249-/-/-/-/,52.359856,4.902558,health_care,NaN,1249
4,Sarphatistraat 14,Kantoorfunctie,516-/-/-/-/,52.359856,4.902558,office,NaN,516
...,...,...,...,...,...,...,...,...
9232,Voormalige Stadstimmertuin 2 K,Woonfunctie,184-/-/-/-/,52.361258,4.905530,house,NaN,184
9233,Voormalige Stadstimmertuin 2 L,Woonfunctie,139-/-/-/-/,52.361189,4.905337,house,NaN,139
9234,Voormalige Stadstimmertuin 2 M,Woonfunctie,169-/-/-/-/,52.361222,4.905546,house,NaN,169
9235,Voormalige Stadstimmertuin 2 N,Woonfunctie,446-/-/-/-/,52.361169,4.905344,house,NaN,446


In [341]:
# For all buildings with an Industrial function (Industriefunctie) smaller than 150 square meters, the function will be changed to a house, otherwise it will become "industrial"
df_addresses.loc[(df_addresses["Function"] == "Industriefunctie") & (df_addresses["Area"] <= 150), "Category"] = "house"
df_addresses.loc[(df_addresses["Function"] == "Industriefunctie") & (df_addresses["Area"] > 150), "Category"] = "industrial"
print(df_addresses["Category"].value_counts())

Category
house               8539
office               311
public_place         100
shop                  67
hotel                 43
                    ... 
secondary_school       1
fitness_centre         1
bar                    1
school                 1
health_centre          1
Name: count, Length: 62, dtype: int64


In [342]:
# Changing the area-column towards NaN if the address does not have "house" as its category
# The area was included due to estimating the amount of people that live in the house, so it is not necessary if it is not a house
df_addresses.loc[df_addresses["Category"] != "house", "Area"] = None
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name,Area
0,Sarphatistraat 4,Kantoorfunctie,760-/-/-/-/,52.359783,4.902103,office,NaN,<NA>
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,office,NaN,<NA>
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,public_place,NaN,<NA>
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249-/-/-/-/,52.359856,4.902558,health_care,NaN,<NA>
4,Sarphatistraat 14,Kantoorfunctie,516-/-/-/-/,52.359856,4.902558,office,NaN,<NA>
...,...,...,...,...,...,...,...,...
9232,Voormalige Stadstimmertuin 2 K,Woonfunctie,184-/-/-/-/,52.361258,4.905530,house,NaN,184
9233,Voormalige Stadstimmertuin 2 L,Woonfunctie,139-/-/-/-/,52.361189,4.905337,house,NaN,139
9234,Voormalige Stadstimmertuin 2 M,Woonfunctie,169-/-/-/-/,52.361222,4.905546,house,NaN,169
9235,Voormalige Stadstimmertuin 2 N,Woonfunctie,446-/-/-/-/,52.361169,4.905344,house,NaN,446


In [343]:
# Creating a new column with the predicted amount of people living in the house by creating a function to calculate this
# The amount of people that live per house divided per square meter distribution:
# 0-25: 0, 25-60: 1, 60-95: 2, 95-110: 3, 110-150: 3.5, 150-200: 4, 200+: 5
def predict_residents(column):
    # Checking if the value is <NA> and if so, returning NaN for this column
    if pd.isna(column):
        return np.nan
    elif column <= 25:
        return 0
    elif column <= 50:
        return 1
    elif column <= 95:
        return 2
    elif column <= 110:
        return 3
    elif column <= 180:
        return 4
    elif column <= 250:
        return 5
    else:
        return 6

# Applying the function to create the new column
df_addresses["pred_residents"] = df_addresses['Area'].apply(predict_residents)
df_addresses

,Address,Function,Size,Latitude,Longitude,Category,Name,Area,pred_residents
0,Sarphatistraat 4,Kantoorfunctie,760-/-/-/-/,52.359783,4.902103,office,NaN,<NA>,NaN
1,Sarphatistraat 6,Kantoorfunctie,1875,52.359801,4.902221,office,NaN,<NA>,NaN
2,Sarphatistraat 8,Bijeenkomstfunctie,1880,52.359820,4.902338,public_place,NaN,<NA>,NaN
3,Sarphatistraat 12,Gezondheidszorgfunctie,1249-/-/-/-/,52.359856,4.902558,health_care,NaN,<NA>,NaN
4,Sarphatistraat 14,Kantoorfunctie,516-/-/-/-/,52.359856,4.902558,office,NaN,<NA>,NaN
...,...,...,...,...,...,...,...,...,...
9232,Voormalige Stadstimmertuin 2 K,Woonfunctie,184-/-/-/-/,52.361258,4.905530,house,NaN,184,5.0
9233,Voormalige Stadstimmertuin 2 L,Woonfunctie,139-/-/-/-/,52.361189,4.905337,house,NaN,139,4.0
9234,Voormalige Stadstimmertuin 2 M,Woonfunctie,169-/-/-/-/,52.361222,4.905546,house,NaN,169,4.0
9235,Voormalige Stadstimmertuin 2 N,Woonfunctie,446-/-/-/-/,52.361169,4.905344,house,NaN,446,6.0


In [344]:
# Deleting unnecessary columns only used for data cleaning and replaced by other columns
columns_delete = ["Function", "Size"]
df_addresses = df_addresses.drop(columns=columns_delete)

df_addresses

,Address,Latitude,Longitude,Category,Name,Area,pred_residents
0,Sarphatistraat 4,52.359783,4.902103,office,NaN,<NA>,NaN
1,Sarphatistraat 6,52.359801,4.902221,office,NaN,<NA>,NaN
2,Sarphatistraat 8,52.359820,4.902338,public_place,NaN,<NA>,NaN
3,Sarphatistraat 12,52.359856,4.902558,health_care,NaN,<NA>,NaN
4,Sarphatistraat 14,52.359856,4.902558,office,NaN,<NA>,NaN
...,...,...,...,...,...,...,...
9232,Voormalige Stadstimmertuin 2 K,52.361258,4.905530,house,NaN,184,5.0
9233,Voormalige Stadstimmertuin 2 L,52.361189,4.905337,house,NaN,139,4.0
9234,Voormalige Stadstimmertuin 2 M,52.361222,4.905546,house,NaN,169,4.0
9235,Voormalige Stadstimmertuin 2 N,52.361169,4.905344,house,NaN,446,6.0


In [345]:
df_addresses.to_csv("scraped_addresses.csv", index=False)